In [3]:
from gen_ai_hub.proxy.core.proxy_clients import set_proxy_version

#use btp proxy llm



In [4]:
from gen_ai_hub.proxy.langchain import init_llm
from hana_ai.vectorstore.embedding_service import GenAIHubEmbeddings
llm = init_llm('gpt-4', temperature=0.0, max_tokens=512)
embed_model = GenAIHubEmbeddings(url='http://ls3085.wdf.sap.corp:7880/embedding')

In [5]:
from llama_index.core.node_parser import SentenceWindowNodeParser
from llama_index.core.node_parser import SentenceSplitter

# create the sentence window node parser w/ default settings
node_parser = SentenceWindowNodeParser.from_defaults(
    window_size=3,
    window_metadata_key="window",
    original_text_metadata_key="original_text",
)

# base node parser is a sentence splitter
text_splitter = SentenceSplitter()

from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = embed_model
Settings.text_splitter = text_splitter

In [6]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(
    input_files=["./hana_sql.pdf"]
).load_data()

In [7]:
nodes = node_parser.get_nodes_from_documents(documents)

In [8]:
base_nodes = text_splitter.get_nodes_from_documents(documents)

In [9]:
from llama_index.core import VectorStoreIndex

sentence_index = VectorStoreIndex(nodes)

In [10]:
base_index = VectorStoreIndex(base_nodes)

In [11]:
from llama_index.core.postprocessor import MetadataReplacementPostProcessor

query_engine = sentence_index.as_query_engine(
    similarity_top_k=2,
    # the target key defaults to `window` to match the node_parser's default
    node_postprocessors=[
        MetadataReplacementPostProcessor(target_metadata_key="window")
    ],
)



In [12]:
window_response = query_engine.query(
    "How can I calculate the max value group by a column?"
)

print(window_response)

C:\Users\I308290\AppData\Local\miniconda3\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `predict_messages` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


To calculate the maximum value grouped by a column, you can use the MAX aggregate function in SQL. The syntax would look something like this: 

SELECT column_name, MAX(value_column) 
FROM table_name 
GROUP BY column_name;

Replace 'column_name' with the name of the column you want to group by, 'value_column' with the name of the column you want to find the maximum value of, and 'table_name' with the name of your table.


In [13]:
window = window_response.source_nodes[0].node.metadata["window"]
sentence = window_response.source_nodes[0].node.metadata["original_text"]

print(f"Window: {window}")
print("------------------")
print(f"Original Sentence: {sentence}")

Window: The example below uses the  ctional MY_TABLE and returns an array with 8 numbers representing the amplitude part (i.e.
 SQRT(REAL^2 + IMAGINARY^2)) of the result.
 The example below uses the  ctional MY_TABLE returns an array with 8 numbers representing the phase part of the result and
ranges between -PI and +PI.
 Related Information
Aggregate Functions
COLLATIONS System View
SERIES_GENERATE Function (Series Data)<order_by_clause>  ::= ORDER BY <order_by_expression>  [, <order_by_expression>  [,...] ]  
 
<order_by_expression>  ::=  
 <column_name>  [ <collate_clause>  ] [ ASC | DESC ] [ NULLS FIRST | NULLS LAST ]  
 | <column_position>  [ <collate_clause>  ] [ ASC | DESC ] [ NULLS FIRST | NULLS LAST ]  
 
<collate_clause>  ::= COLLATE <collation_name>
SELECT DFT(FRACTION_OF_MIN_MAX_RANGE, 4 SERIES(EQUIDISTANT INCREMENT BY 1 PERIOD FOR SERIES(element
    FROM SERIES_GENERATE_INTEGER(1,0,10);
SELECT DFT(col, 4 ORDER BY DATE).REAL FROM MY_TABLE;
SELECT DFT(col, 8 ORDER BY DATE).I